<a href="https://colab.research.google.com/github/SFDCsid/APEX-CLASS-TEST-CLASS-/blob/main/FYERS_LOGIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. WORKING CODE WITH FORCE TOKEN GENERATION
# Latest code for fyers api authentication to get access token

# Install Fyers API SDK
!pip install fyers-apiv3 --quiet

import json
import pytz
import datetime
from fyers_apiv3 import fyersModel
from google.colab import drive
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import os

# ====== USER CONFIG ======
client_id = "BEJLQY7YI5-100"         # Replace with your APP_ID
secret_key = "TC3HVJ8KZ8"        # Replace with your secret key
redirect_uri = "https://example.com" # Replace with your redirect URI
response_type = "code"
grant_type = "authorization_code"

# ====== MOUNT GOOGLE DRIVE ======
drive.mount('/content/drive')
folder_path = "/content/drive/MyDrive/FyersToken"
os.makedirs(folder_path, exist_ok=True)
token_txt_path = f"{folder_path}/access_token.txt"
token_json_path = f"{folder_path}/token_details.json"

# ====== IST TIME HELPERS ======
def ist_now():
    return datetime.datetime.now(pytz.timezone("Asia/Kolkata"))

def format_ist(dt):
    return dt.strftime("%A, %d %B %Y, %I:%M %p IST")

# ====== SAVE TOKEN ======
def save_token(access_token):
    created_time = ist_now()
    expiry_time = created_time + datetime.timedelta(days=1)
    # Save TXT
    with open(token_txt_path, "w") as f:
        f.write(access_token)
    # Save JSON
    details = {
        "access_token": access_token,
        "created_time": created_time.isoformat(),
        "expiry_time": expiry_time.isoformat()
    }
    with open(token_json_path, "w") as f:
        json.dump(details, f, indent=4)
    return created_time, expiry_time

# ====== LOAD TOKEN ======
def load_token():
    if os.path.exists(token_txt_path):
        with open(token_txt_path, "r") as f:
            return f.read().strip()
    return None

# ====== CHECK TOKEN VALIDITY ======
def check_token_and_get_profile(token):
    try:
        fyers = fyersModel.FyersModel(client_id=client_id, token=token, log_path="")
        profile = fyers.get_profile()
        if profile.get("s") == "ok":
            print("🟦 Token is valid — Profile fetched successfully")
            print(profile)
            return True
        else:
            print("🟥 Token invalid — Profile fetch failed")
            print(profile)
            return False
    except Exception as e:
        print("🟥 Token check failed with error:", e)
        return False

# ====== GENERATE NEW TOKEN (with textbox) ======
def generate_new_token():
    session = fyersModel.SessionModel(
        client_id=client_id,
        secret_key=secret_key,
        redirect_uri=redirect_uri,
        response_type=response_type,
        grant_type=grant_type
    )
    auth_url = session.generate_authcode()
    print("\n🔗 Open this URL to log in and authorize:\n", auth_url)

    # Textbox + submit button
    url_box = widgets.Text(
        placeholder='Paste FULL redirected URL here',
        description='URL:',
        layout=widgets.Layout(width='100%')
    )
    submit_button = widgets.Button(
        description='Generate Token',
        button_style='success',
        icon='check'
    )
    output_area = widgets.Output()

    def on_submit(b):
        with output_area:
            clear_output()
            redirected_url = url_box.value.strip()
            try:
                auth_code = redirected_url.split("auth_code=")[-1].split("&")[0]
            except:
                print("❌ Could not extract auth_code from URL. Paste full redirected URL correctly.")
                return
            session.set_token(auth_code)
            response = session.generate_token()
            access_token = response.get("access_token")
            if not access_token:
                print("❌ Failed to generate access token. Response:", response)
                return
            # Save token (overwrite existing)
            created_time, expiry_time = save_token(access_token)
            if check_token_and_get_profile(access_token):
                print(f"🔑 Access Token: {access_token}")
                time_left = expiry_time - ist_now()
                print(f"📅 Created: {format_ist(created_time)}")
                print(f"⏳ Expires: {format_ist(expiry_time)}")
                print(f"⏰ Time left: {time_left.seconds//3600} hours {((time_left.seconds//60)%60)} minutes")

    submit_button.on_click(on_submit)
    display(url_box, submit_button, output_area)

# ====== MAIN FLOW ======
access_token = load_token()
token_still_valid = False

if access_token and check_token_and_get_profile(access_token):
    token_still_valid = True
    print(f"🔑 Access Token: {access_token}")
    with open(token_json_path, "r") as f:
        details = json.load(f)
    created_time = datetime.datetime.fromisoformat(details["created_time"])
    expiry_time = datetime.datetime.fromisoformat(details["expiry_time"])
    created_time = created_time.astimezone(pytz.timezone("Asia/Kolkata"))
    expiry_time = expiry_time.astimezone(pytz.timezone("Asia/Kolkata"))
    time_left = expiry_time - ist_now()
    print(f"📅 Created: {format_ist(created_time)}")
    print(f"⏳ Expires: {format_ist(expiry_time)}")
    print(f"⏰ Time left: {time_left.seconds//3600} hours {((time_left.seconds//60)%60)} minutes")
else:
    print("🔑 No valid token found. Please log in to generate one.")
    generate_new_token()

# ====== GOOGLE DRIVE LINK ======
drive_folder_url = "https://drive.google.com/drive/u/2/folders/1zhZhQUKRnkorn8dbrwu5gT2OAy1SS3aC"
display(HTML(f'<a href="{drive_folder_url}" target="_blank">📂 Open Token Folder in Google Drive</a>'))

display(HTML('🔗APP link: <a href="https://myapi.fyers.in/dashboard" target="_blank">https://myapi.fyers.in/dashboard</a>'))

print("\n")  # adds a blank line

# ====== FORCE GENERATE TOKEN BUTTON ======
force_token_button = """
<form action="#" method="get">
    <input type="submit" value="🔄 Force Generate Token"
           onclick="google.colab.kernel.invokeFunction('forceToken', [], {}); return false;">
</form>
"""
display(HTML(force_token_button))

# ====== REGISTER JS CALLBACK ======
from google.colab import output
def force_token_callback():
    print("\n⚠️ Force token generation started...")
    generate_new_token()

output.register_callback('forceToken', force_token_callback)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 6.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires reques

Text(value='', description='URL:', layout=Layout(width='100%'), placeholder='Paste FULL redirected URL here')

Button(button_style='success', description='Generate Token', icon='check', style=ButtonStyle())

Output()

In [ ]:
#2. code for fyers api authentication to get access token

# Install Fyers API SDK
!pip install fyers-apiv3 --quiet

import json
import pytz
import datetime
from fyers_apiv3 import fyersModel
from google.colab import drive
from IPython.display import HTML

# ====== USER CONFIG ======
client_id = "BEJLQY7YI5-100"         # Replace with your APP_ID
secret_key = "TC3HVJ8KZ8"        # Replace with your secret key
redirect_uri = "https://example.com" # Replace with your redirect URI
response_type = "code"
grant_type = "authorization_code"

# ====== MOUNT GOOGLE DRIVE ======
drive.mount('/content/drive')
folder_path = "/content/drive/MyDrive/FyersToken"

import os
os.makedirs(folder_path, exist_ok=True)
token_txt_path = f"{folder_path}/access_token.txt"
token_json_path = f"{folder_path}/token_details.json"

# ====== IST TIME HELPERS ======
def ist_now():
    return datetime.datetime.now(pytz.timezone("Asia/Kolkata"))

def format_ist(dt):
    return dt.strftime("%A, %d %B %Y, %I:%M %p IST")

# ====== SAVE TOKEN ======
def save_token(access_token):
    created_time = ist_now()
    expiry_time = created_time + datetime.timedelta(days=1)
    # Save TXT (token only)
    with open(token_txt_path, "w") as f:
        f.write(access_token)
    # Save JSON (all details)
    details = {
        "access_token": access_token,
        "created_time": created_time.isoformat(),
        "expiry_time": expiry_time.isoformat()
    }
    with open(token_json_path, "w") as f:
        json.dump(details, f, indent=4)
    return created_time, expiry_time

# ====== LOAD TOKEN IF EXISTS ======
def load_token():
    if os.path.exists(token_txt_path):
        with open(token_txt_path, "r") as f:
            return f.read().strip()
    return None

# ====== CHECK TOKEN VALIDITY ======
def check_token_and_get_profile(token):
    try:
        fyers = fyersModel.FyersModel(client_id=client_id, token=token, log_path="")
        profile = fyers.get_profile()
        if profile.get("s") == "ok":
            print("🟦 Token is valid — Profile fetched successfully")
            print(profile)
            return True
        else:
            print("🟥 Token invalid — Profile fetch failed")
            print(profile)  # Show error response
            return False
    except Exception as e:
        print("🟥 Token check failed with error:", e)
        return False

# ====== MAIN FLOW ======
access_token = load_token()
token_still_valid = False

if access_token and check_token_and_get_profile(access_token):
    token_still_valid = True
    print(f"🔑 Access Token: {access_token}")
    with open(token_json_path, "r") as f:
        details = json.load(f)
    created_time = datetime.datetime.fromisoformat(details["created_time"])
    expiry_time = datetime.datetime.fromisoformat(details["expiry_time"])
    created_time = created_time.astimezone(pytz.timezone("Asia/Kolkata"))
    expiry_time = expiry_time.astimezone(pytz.timezone("Asia/Kolkata"))
    time_left = expiry_time - ist_now()

    print(f"📅 Created: {format_ist(created_time)}")
    print(f"⏳ Expires: {format_ist(expiry_time)}")
    print(f"⏰ Time left: {time_left.seconds//3600} hours {((time_left.seconds//60)%60)} minutes")

else:
    print("🔑 No valid token found. Please log in to generate one.")
    print(f"🔑 Access Token: {access_token}")
    session = fyersModel.SessionModel(
        client_id=client_id,
        secret_key=secret_key,
        redirect_uri=redirect_uri,
        response_type=response_type,
        grant_type=grant_type
    )
    auth_url = session.generate_authcode()
    print("🔗 Open this URL to log in and authorize:\n", auth_url)

    auth_code = input("Paste the redirected URL here: ").split("auth_code=")[-1].split("&")[0]
    session.set_token(auth_code)
    response = session.generate_token()
    access_token = response["access_token"]

    created_time, expiry_time = save_token(access_token)

    if check_token_and_get_profile(access_token):
        print(f"🔑 Access Token: {access_token}")
        time_left = expiry_time - ist_now()
        print(f"📅 Created: {format_ist(created_time)}")
        print(f"⏳ Expires: {format_ist(expiry_time)}")
        print(f"⏰ Time left: {time_left.seconds//3600} hours {((time_left.seconds//60)%60)} minutes")

# ====== SHOW GOOGLE DRIVE LINK ======
drive_folder_url = "https://drive.google.com/drive/u/2/folders/1zhZhQUKRnkorn8dbrwu5gT2OAy1SS3aC"
HTML(f'<a href="{drive_folder_url}" target="_blank">📂 Open Token Folder in Google Drive</a>')

=======       ACCESS TOKEN GENERATION CODE ENDS HERE     ========